In [1]:
# PACKAGES
using Distributions, HypothesisTests, Plots, Optim, Statistics, CSV, DataFrames, Extremes, Printf, XLSX, Psychro, Base.Threads, JLD2, GLM, Impute

# IMPORT DATASET
name_MasterFile = "5 minute masterdata_Full_Sep 2023.xlsx"
sheetName = "5 min data 2022_2023"
df = XLSX.readdata(name_MasterFile,sheetName,"A8:DA85135") |> x -> DataFrame(x, :auto);


In [2]:
# CLEANUP DATASET

#Rename columns
rename!(df, :x2 => :Date, :x3 => :Time, :x8 => :ṁ_HeatExchanger1, :x10 => :samplingTime_HeatExchanger1, :x12 => :LMTD_HeatExchanger1, :x17 => :ṁ_HeatExchanger2, :x19 => :samplingTime_HeatExchanger2, :x21 => :LMTD_HeatExchanger2, :x26 => :ṁ_Chiller, :x28 => :samplingTime_Chiller, :x30 => :LMTD_Chiller, :x32 => :I_Pump1, :x34 => :samplingTime_Pump1, :x37 => :I_Pump2, :x39 => :samplingTime_Pump2, :x42 => :I_Pump3, :x44 => :samplingTime_Pump3);
rename!(df, :x80 => :T_CT1, :x81 => :T_CT2, :x82 => :T_CT3, :x92 => :T_amb, :x94 => :RH_amb, :x98 => :P_amb);

#Add Vortisand energy
E_vortisand = fill(0.31, size(df)[1]); #Vortisand energy consumption (KWh)
E_vortisand[33695:65948] .= 0;
E_vortisand[70286:78955] .= 0;
df.E_vortisand = E_vortisand;

#Add wetbulb temperature
T_wb = Psychro.wetbulb.(MoistAir, df.T_amb.+273.15, RelHum, df.RH_amb./100, df.P_amb.*1000).-273.15; #Wetbulb temperature - [°C]
df.T_wb = T_wb;

In [3]:
#Missing data
#Replace values with previous
for col in [:ṁ_HeatExchanger1, :LMTD_HeatExchanger1, :ṁ_HeatExchanger2, :LMTD_HeatExchanger2, :ṁ_Chiller, :LMTD_Chiller,:I_Pump1]
    Impute.locf!(df[!, col])
end
#Replace values with average of non missing values
for col in [:x48, :x49, :x50, :x51, :x52, :x53, :x54, :x55, :x56, :x57, :x58, :x59, :x60, :x61, :x62, :x63, :x64, :x65, :x66, :x67, :x68, :x69, :x70, :x71, :x72, :x73, :x74, :x75, :x76, :x77, :x78]
    replace!(df[!, col], missing => mean(skipmissing(df[!, col][1:33694])))
end


In [4]:
#Outliers and negative values
for col in [:ṁ_HeatExchanger1, :LMTD_HeatExchanger1, :ṁ_HeatExchanger2, :LMTD_HeatExchanger2, :ṁ_Chiller, :LMTD_Chiller,:I_Pump1]
    df[!, col] .= max.(df[!, col], 0)
end

#Function for the outliers
for col in [:LMTD_HeatExchanger1, :LMTD_HeatExchanger2, :LMTD_Chiller]
    for i in 2:size(df, 1)
        if df[i, col] > 100
            df[i, col] = df[i-1, col]
        end
    end
end    

In [15]:
#Information on the dataset
describe(df) #OUtputs useful information on the dataframe
#describe(df).nmissing[71:90] #Outputs the number of missing fields in column number [n:m]
describe(df).min[14:35]

22-element Vector{Any}:
  19.1
  16.3
  16.2
   0.0
 -40.2121769230769
   0
   0.0
   0.0
  14.3
    nothing
   6.9
   4.6
   0.0
   0.0
   0
   0.0
   0.0
   0.0
   0.1
   0.0
   0.0
   0.0

In [12]:
df

Row,x1,Date,Time,x4,x5,x6,x7,ṁ_HeatExchanger1,x9,samplingTime_HeatExchanger1,x11,LMTD_HeatExchanger1,x13,x14,x15,x16,ṁ_HeatExchanger2,x18,samplingTime_HeatExchanger2,x20,LMTD_HeatExchanger2,x22,x23,x24,x25,ṁ_Chiller,x27,samplingTime_Chiller,x29,LMTD_Chiller,x31,I_Pump1,x33,samplingTime_Pump1,x35,x36,I_Pump2,x38,samplingTime_Pump2,x40,x41,I_Pump3,x43,samplingTime_Pump3,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,T_CT1,T_CT2,T_CT3,x83,x84,x85,x86,x87,x88,x89,x90,x91,T_amb,x93,RH_amb,x95,x96,x97,P_amb,x99,x100,⋯
,Any,Any,Any,Any,Any,Any,Any,Real,Any,Any,Any,Float64,Any,Any,Any,Any,Real,Any,Any,Any,Float64,Any,Any,Any,Any,Float64,Any,Any,Any,Float64,Any,Real,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,⋯
1,missing,12-May-22,10:00:00 A,40.8,33.9,21.7,25.1,0.0595083,845.291,5,70.4409,13.8765,41.1,25.7,21.7,28.5,0.00697222,0.0,0,0,7.49519,25.9,22.6,11.3,11.3,5.19e-5,0.0,0,0,12.8796,70.4409,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,23.8,20.3847,5,1.69872,0.643243,1.69872,2.79144,2.62187,2.63888,2.52719,2.49963,15.8559,3.68051,3.4306,3.6217,3.39986,3.60032,4.18567,0.231453,0.217324,0.218536,0.209087,0.206852,1.32119,2.70665,2.58303,9.17779,0.118518,0.117026,0.129767,0.448777,0.427622,1.52804,4.82249,0.365311,2.40444,0.310833,2.00956,22.3,22.5,26.2,2.8,6.0,3.6,7.9282,8.1282,11.8282,0.260994,0.430781,0.233339,24.7,5.9,30,0,6,2,101.78,14.3718,35.053,⋯
2,missing,12-May-22,10:05:00 A,40.8,33.9,24.1,27.2,0.0620139,803.156,5,66.9297,11.5964,41.1,25.7,24.1,28.5,0.00755556,0.0,0,0,5.33025,26.0,22.6,11.3,11.3,0.000285483,0.0,0,0,12.9256,66.9297,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,26.0,22.269,5,1.85575,0.702703,1.85575,2.79144,2.62187,2.63888,2.52719,2.49963,15.8559,3.68051,3.4306,3.6217,3.39986,3.60032,4.18567,0.231453,0.217324,0.218536,0.209087,0.206852,1.32119,2.70665,2.58303,9.17779,0.118518,0.117026,0.129767,0.448777,0.427622,1.52804,4.82249,0.365311,2.40444,0.310833,2.16658,22.3,22.5,26.2,4.9,6.0,3.6,7.9282,8.1282,11.8282,0.381971,0.430781,0.233339,24.7,5.9,30,0,6,2,101.78,14.3718,30.8918,⋯
3,missing,12-May-22,10:10:00 A,40.8,33.9,26.1,29.1,0.0615528,771.469,5,64.2891,9.61858,41.1,25.7,26.1,28.4,0.00808333,0.0,0,0,3.78843,25.9,22.7,11.3,11.3,0.0,0.0,0,0,12.9341,64.2891,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,30.1,25.7806,5,2.14839,0.813514,2.14839,2.79144,2.62187,2.63888,2.52719,2.49963,15.8559,3.68051,3.4306,3.6217,3.39986,3.60032,4.18567,0.231453,0.217324,0.218536,0.209087,0.206852,1.32119,2.70665,2.58303,9.17779,0.118518,0.117026,0.129767,0.448777,0.427622,1.52804,4.82249,0.365311,2.40444,0.310833,2.45922,22.3,22.5,26.2,6.8,5.9,3.5,7.9282,8.1282,11.8282,0.461699,0.426664,0.228337,24.7,5.9,30,0,6,2,101.78,14.3718,26.1421,⋯
4,missing,12-May-22,10:15:01 A,40.8,33.9,28.0,31.1,0.0558472,723.291,5,60.2742,7.64321,41.1,25.7,28.0,28.4,0.00863889,0.0,0,0,8.77864,25.8,22.7,11.3,11.3,7.785e-5,0.0,0,0,12.8879,60.2742,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,28.8,24.6672,5,2.0556,0.778378,2.0556,2.79144,2.62187,2.63888,2.52719,2.49963,15.8559,3.68051,3.4306,3.6217,3.39986,3.60032,4.18567,0.231453,0.217324,0.218536,0.209087,0.206852,1.32119,2.70665,2.58303,9.17779,0.118518,0.117026,0.129767,0.448777,0.427622,1.52804,4.82249,0.365311,2.40444,0.310833,2.36643,22.3,22.5,26.2,8.8,5.9,3.5,7.9282,8.1282,11.8282,0.526058,0.426664,0.228337,24.7,5.9,30,0,6,2,101.78,14.3718,25.4705,⋯
5,missing,12-May-22,10:20:00 A,40.8,33.9,27.0,30.4,0.0591667,840.438,5,70.0365,8.53067,41.1,25.7,27.0,28.4,0.00844444,0.0,0,0,6.1424,25.9,22.7,11.3,11.3,5.19e-5,0.0,0,0,12.9341,70.0365,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,28.9,24.7528,5,2.06274,0.781081,2.06274,2.79144,2.62187,2.63888,2.52719,2.49963,15.8559,3.68051,3.4306,3.6217,3.39986,3.60032,4.18567,0.231453,0.217324,0.218536,

In [25]:
# VARIABLE CALCULATIONS

#Power and energy calculations
ρ = 999; #Water density (Kg/m^3)
cp = 4182; #Specific heat capacity of water (J/kg*°C)
V = 575; #Voltage of the pumps (V)

#Heat exchanger 1 (500-EEE-01)
P_HeatExchanger1 = ρ.*df.ṁ_HeatExchanger1.*cp.*df.LMTD_HeatExchanger1./1000; #Power obtained from heat exchanger 1 - [KW]
E_HeatExchanger1 = P_HeatExchanger1.*df.samplingTime_HeatExchanger1/60; #Energy obtained from heat exchanger 1 - [KWh]
#Heat exchanger 2 (500-EEE-03)
P_HeatExchanger2 = ρ.*df.ṁ_HeatExchanger2.*cp.*df.LMTD_HeatExchanger2/1000; #Power obtained from heat exchanger 2 - [KW]
E_HeatExchanger2 = P_HeatExchanger2.*df.samplingTime_HeatExchanger2/60; #Energy obtained from heat exchanger 2 - [KWh]
#Chiller (500-REF-01)
P_Chiller = ρ.*df.ṁ_Chiller.*cp.*df.LMTD_Chiller/1000; #Power obtained from chiller - [KW]
E_Chiller = P_Chiller.*df.samplingTime_Chiller/60; #Energy obtained from chiller - [KWh]
#Pump 1 (500-PET-01)
P_Pump1 = V.*df.I_Pump1/1000; #Power needed for pump 1 - [KW]
E_Pump1 = P_Pump1.*df.samplingTime_Pump1/60; #Energy needed for pump 1 - [KWh]
#Pump 2 (500-PET-02)
P_Pump2 = V.*df.I_Pump2/1000; #Power needed for pump 2 - [KW]
E_Pump2 = P_Pump2.*df.samplingTime_Pump2/60; #Energy needed for pump 2 - [KWh]
#Pump 3 (500-PET-03)
P_Pump3 = V.*df.I_Pump3/1000; #Power needed for pump 3 - [KW]
E_Pump3 = P_Pump3.*df.samplingTime_Pump3/60; #Energy needed for pump 3 - [KWh]
#Fans CT1 (500-TOE-01)
P_Fan1 = df[:,48]+df[:,49]; #Power needed for fans of cooling tower 1 - [KW]
E_Fan1 = df[:,48].*df[:,54]+df[:,49].*df[:,55]; #Energy needed for fans of cooling tower 1 - [KWh]
#Fans CT2 (500-TOE-02)
P_Fan2 = df[:,50]+df[:,51]; #Power needed for fans of cooling tower 2 - [KW]
E_Fan2 = df[:,50].*df[:,56]+df[:,51].*df[:,57]; #Energy needed for fans of cooling tower 2 - [KWh]
#Fans CT3 (500-TOE-03)
P_Fan3 = df[:,52]+df[:,53]; #Power needed for fans of cooling tower 2 - [KW]
E_Fan3 = df[:,52].*df[:,58]+df[:,53].*df[:,59]; #Energy needed for fans of cooling tower 2 - [KWh]

# Calculation of the thermal load demand on the system
# Assuming that Heat exchanger 1 is related to CT1 and Heat exchanger 2 is related to CT2 and CT3 and Chiller is related to CT3
df.Load_tot = P_HeatExchanger1.+P_HeatExchanger2.+P_Chiller; #Total thermal load - [KW]

# Calculation of the coeffecient of performance (COP)
df.E_exchanged = E_HeatExchanger1.+E_HeatExchanger2.+E_Chiller; #Total amount of energy available for the heat exchange - [KWh]
df.E_consumed = E_Pump1.+E_Pump2.+E_Pump3.+E_Fan1.+E_Fan2.+E_Fan3+E_vortisand; #Total amount of energy consumed to make the system work - [KWh]
df.COP = df.Load_tot./df.E_consumed; #E_exchanged/E_consumed
replace!(df.COP, NaN => 0);
replace!(df.COP, Inf => 0);

In [26]:
#Get rid of unnecessary columns
df_x = select(df,[:T_wb,:T_CT1,:T_CT2,:T_CT3,:Load_tot,:E_exchanged,:E_consumed,:COP]);

In [27]:
#Generate datasets for ON/OFF
df_ON = [df_x[1:33694,:];df_x[70286:78955,:]];
df_OFF = [df_x[33695:65948,:];df_x[65949:70285,:];df_x[78956:end,:]];
df_clean = select(df,[:Date,:Time,:T_wb,:T_CT1,:T_CT2,:T_CT3,:Load_tot,:E_exchanged,:E_consumed,:COP]);

In [30]:
describe(df_clean)
describe(df_ON)
#describe(df_OFF)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Union…,Union…,Union…,Int64,DataType
1,T_wb,15.7243,-10.5147,16.4699,25.3792,0,Float64
2,T_CT1,22.7421,7.4,22.5,37.6,0,Any
3,T_CT2,,,,,0,Any
4,T_CT3,24.8938,14.1,24.1,40.8,0,Any
5,Load_tot,2822.78,0.0,2189.73,8359.67,0,Float64
6,E_exchanged,191.033,0.0,150.307,658.373,0,Float64
7,E_consumed,131.629,0.00479167,114.346,456.305,0,Float64
8,COP,155.022,0.0,33.762,22157.5,0,Float64


In [29]:
#SAVE CLEANED UP DATA
CSV.write("df_clean.csv", df_clean)
CSV.write("df_ON.csv", df_ON)
CSV.write("df_OFF.csv", df_OFF)

"df_OFF.csv"

In [ ]:
#INPUT CLEANED UP DATA
df_clean = CSV.read("df_clean.csv")
df_ON = CSV.read("df_ON.csv")
df_OFF = CSV.read("df_OFF.csv")